## Dimension reduction of BERT SOTU embeddings

Run this notebook in the root directory (where SOTU files can be accessed) to replicate dimension reduction algorithms (UMAP, TriMAP, PaCMAP) applied to the embedding of SOTU presidential addresses via DistilBERT.  

Another part of the notebook is about authorship attribution by using nearest neighbors in the original DistilBERT embedding. 

This notebook can be run on a CPU instance.

# Initializing

In [ ]:
!pip -q install tensorflow transformers faiss-cpu umap-learn trimap pacmap matplotlib pandas seaborn

# Imports

In [ ]:
import tensorflow as tf
from transformers import BertTokenizer, TFDistilBertModel
import matplotlib.pyplot as plt
import numpy as np
import faiss
import umap, trimap, pacmap
import os
import pickle
import pandas as pd
import seaborn as sns
from sklearn.cluster import KMeans

# Model and dataset

Chunk length and shift for the sliding window

In [ ]:
WINDOW_LEN, WINDOW_SHIFT = 256, 64

Using DistilBERT (faster and more lightweight than BERT) 

In [ ]:
#
# Load the BERT tokenizer
#
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
#
# Load the DistilBERT model
#
bert_model = TFDistilBertModel.from_pretrained("distilbert-base-uncased")
#
# Define a function to generate embeddings (normalized to unit norm)
#
def generate_embedding(text, window_len, window_shift, preprocessor, model):
    # Split the text into words
    words = text.split()
    #
    # Integrator for word strings
    integrator = 0
    #
    # Iterate through windows with the specified shift
    for i in range(0, len(words) - window_len + 1, window_shift):
        string = ' '.join(words[i:i + window_len])
        inputs = preprocessor(string, return_tensors='tf', max_length=512, truncation=True, padding='max_length')['input_ids']

        outputs = model(inputs)
        last_hidden_states = outputs.last_hidden_state

        avg_pool = tf.reduce_mean(last_hidden_states, axis=1)
        integrator += avg_pool[0]
        print(integrator.shape)
    #
    # Include the last window if it's shorter than window_len
    last_window_start = len(words) % window_len
    if last_window_start > 0:
        string = ' '.join(words[-last_window_start:])
        inputs = preprocessor(string, return_tensors='tf', max_length=512, truncation=True, padding='max_length')['input_ids']

        outputs = model(inputs)
        last_hidden_states = outputs.last_hidden_state

        avg_pool = tf.reduce_mean(last_hidden_states, axis=1)
        integrator += avg_pool[0]
        print(integrator.shape)
    #
    l2_norm    = tf.norm(integrator, ord='euclidean')
    #
    return integrator/l2_norm

In [ ]:
#
# Function to read and store text files from a directory
# that are label='train', label='test', or both kinds if label=None
#
def store_text_files(directory, index, filenames, window_len, window_shift, preprocessor, model, label=None):
    #
    assert label in ['train', 'test', None]
    #
    end = {'train': ".train.txt", 'test': ".test.txt", None: ".txt"}
    #
    for filename in os.listdir(directory):
        #
        if filename.endswith(end[label]):
            #
            filepath = os.path.join(directory, filename)
            #
            with open(filepath, "r", encoding="utf-8") as file:
                print(f'Acquiring file {filename}')
                text = file.read()
                embedding = generate_embedding(text, window_len, window_shift, preprocessor, model)
                index.add(np.expand_dims(embedding, axis=0))
                filenames.append(filename)

In [ ]:
#
# Function to read and store text files from directories starting with "sotu_"
#
def store_text_files_in_sotu_directories(root_directory, index, filenames, window_len, window_shift, preprocessor, model, label=None):
    #
    for entry in os.listdir(root_directory):
        #
        full_path = os.path.join(root_directory, entry)
        #
        if os.path.isdir(full_path) and entry.startswith("sotu_"):
            store_text_files(full_path, index, filenames, window_len, window_shift, preprocessor, model, label)

In [ ]:
#
# Function to look for the nearest vector in Faiss index
#
def compare_embedding_to_index(file_path, index, filenames, window_len, window_shift, preprocessor, model, top_k=5):
    #
    with open(file_path, "r", encoding="utf-8") as file:
        #
        query_text = file.read()
        query_embedding = generate_embedding(query_text, window_len, window_shift, preprocessor, model)
        #
        # Convert query embedding to a NumPy array
        query_embedding_np = np.array([query_embedding])
        #
        # Search for the closest vectors in the Faiss index
        similarity, indices = index.search(query_embedding_np, k=top_k)
        #
        # Retrieve the top k closest texts from the filenames list
        closest_texts = [filenames[int(x)] for x in indices[0]]
        #
        return closest_texts, similarity

In [ ]:
def test_text_files(directory, index, filenames, top_k=5):
    #
    top_k_files = []
    #
    for filename in os.listdir(directory):
        #
        if filename.endswith('.test.txt'):
            #
            assert (filename in filenames)
            #
            i = filenames.index(filename)
            query = np.array([index.reconstruct(i)])
            similarity, indices = index.search(query, k=top_k+1)
            closest_texts = [filenames[int(x)] for x in indices[0]]
            top_k_list = [closest_texts[1:], similarity[0][1:]]
            top_k_files.append(top_k_list)
    #
    return top_k_files

In [ ]:
#
# Function to read and test the FAISS index on test files from directories starting with "sotu_"
#
def test_text_files_in_sotu_directories(root_directory, index, filenames, top_k=5):
    #
    test = {}
    #
    for entry in os.listdir(root_directory):
        #
        full_path = os.path.join(root_directory, entry)
        #
        if os.path.isdir(full_path) and entry.startswith("sotu_"):
            top_k_entries = test_text_files(full_path,
                                            index,
                                            filenames,
                                            top_k)
            test[entry] = top_k_entries
    #
    return test

# FAISS index

In [ ]:
# BERT embedding dimension
index_dimension = 768
#
# Create FAISS index
index = faiss.IndexFlatIP(index_dimension)
#
# File names
filenames  = []
#
# Sanity check: should be 768, 0, 0, and the index is trained (by default)
#
print(f'Index dimension {index_dimension}, number of entries {index.ntotal}, number of files {len(filenames)}')
print(f'Is trained? {index.is_trained}')

In [ ]:
PREP  = bert_tokenizer
MODEL = bert_model

Computing the BERT embedding and storing it in the index (uncomment to replicate)

In [ ]:
"""
#
# Loading files into the FAISS index with DistilBERT embedding
#
store_text_files_in_sotu_directories(".", index, filenames, WINDOW_LEN, WINDOW_SHIFT, PREP, MODEL, label=None)
faiss.write_index(index, "sotu_bert_complete.faiss")
with open("sotu_bert_filenames_complete.pickle", 'wb') as file:
    pickle.dump(filenames, file)
    file.close()
""";

Loading the index and filenames from the stored dataset

In [ ]:
index = faiss.read_index("sotu_bert_complete.faiss")
with open("sotu_bert_filenames_complete.pickle", 'rb') as file:
    filenames = pickle.load(file)
    file.close()

In [ ]:
#
# Sanity check: should be 768, 229, 229, and the index is trained (by default)
#
print(f'Index dimension {index_dimension}, number of entries {index.ntotal}, number of files {len(filenames)}')
print(f'Is trained? {index.is_trained}')

# Dimension reduction

Dimension reduction and clustering: BERT mapped into dimensions 2 and 3 with UMAP, TriMAP, PaCMAP

In [ ]:
#
# Recover the BERT embedding
#
bert_embedding = []
for i in range(index.ntotal):
    vec = index.reconstruct(i)
    bert_embedding += [vec]

In [ ]:
#
# UMAP the BERT embedding into 2D
#
reducer_umap2 = umap.UMAP(n_neighbors=int(np.sqrt(index_dimension)),
                          n_components=2,
                          metric='cosine',
                          verbose=1)
umap_2_embedding = reducer_umap2.fit_transform(bert_embedding)

In [ ]:
#
# Scatter plot in 2D
#
def plot_2d_embedding(embedding, emb_name):
    labels_year = [int(s.split('_')[1].split('.')[0]) for s in filenames]
    scatter = plt.scatter(embedding[:, 0], embedding[:, 1], c=labels_year)
    plt.title(f'{emb_name} of SOTU in 2D')
    plt.xlabel("X-axis")
    plt.ylabel("Y-axis")
    cbar = plt.colorbar(scatter, label="Year")
    plt.show()
#
plot_2d_embedding(umap_2_embedding, "UMAP")

In [ ]:
#
# TriMAP the BERT embedding into 2D
#
reducer_trimap2 = trimap.TRIMAP(n_dims=2,
                                distance='cosine',
                                verbose=1)
trimap_2_embedding = reducer_trimap2.fit_transform(np.array(bert_embedding))
#
plot_2d_embedding(trimap_2_embedding, "TriMAP")

In [ ]:
#
# PaCMAP the BERT embedding into 2D
#
reducer_pacmap2 = pacmap.PaCMAP(n_neighbors=int(np.sqrt(index_dimension)),
                                n_components=2,
                                distance='angular',
                                verbose=1)
pacmap_2_embedding = reducer_pacmap2.fit_transform(bert_embedding)
#
plot_2d_embedding(pacmap_2_embedding, "PaCMAP")

In [ ]:
#
# UMAP the BERT embedding into 3D
#
reducer_umap3 = umap.UMAP(n_neighbors=int(np.sqrt(index_dimension)),
                          n_components=3,
                          metric='cosine',
                          verbose=1)
umap_3_embedding = reducer_umap3.fit_transform(bert_embedding)

In [ ]:
#
# Scatter plot in 3D
#
def plot_3d_embedding(embedding, emb_name):
    labels_year = [int(s.split('_')[1].split('.')[0]) for s in filenames]
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    scatter = ax.scatter(embedding[:, 0],
                         embedding[:, 1],
                         embedding[:, 2],
                         c=labels_year)
    ax.set_title(f'{emb_name} of SOTU in 3D')
    ax.set_xlabel("X-axis")
    ax.set_ylabel("Y-axis")
    ax.set_zlabel("Z-axis")
    fig.colorbar(scatter, label="Year")
    plt.show()
#
plot_3d_embedding(umap_3_embedding, "UMAP")

In [ ]:
#
# TriMAP the BERT embedding into 3D
#
reducer_trimap3 = trimap.TRIMAP(n_dims=3,
                                distance='cosine',
                                verbose=1)
trimap_3_embedding = reducer_trimap3.fit_transform(np.array(bert_embedding))
#
plot_3d_embedding(trimap_3_embedding, "TriMAP")

In [ ]:
#
# PaCMAP the BERT embedding into 3D
#
reducer_pacmap3 = pacmap.PaCMAP(n_neighbors=int(np.sqrt(index_dimension)),
                                n_components=3,
                                distance='angular',
                                verbose=1)
pacmap_3_embedding = reducer_pacmap3.fit_transform(bert_embedding)
#
plot_3d_embedding(pacmap_3_embedding, "PaCMAP")

# Diagrams for temporal clustering of SOTU speeches

In [ ]:
#
# k-Mean clustering on given embedding with k=2, and then plot the diagram
#
def plot_clustering_diagram(embedding):
    
    kmeans = KMeans(n_clusters=2, random_state=42)
    
    cluster_labels = kmeans.fit_predict(embedding)
    labels_year = [int(s.split('_')[1].split('.')[0]) for s in filenames]
    
    df_emb = pd.DataFrame(data=np.array([labels_year, cluster_labels]).T,
                          columns=['Year', 'Cluster'])
    df_emb = df_emb.sort_values('Year')
    
    # Create a color map based on the values in the 'Cluster' column
    colors = df_emb['Cluster'].map({0: 'blue', 1: 'red'})

    # Same height bars for all years, only colors are different
    df_emb['Value'] = 1.0

    # Plotting a bar graph with customized colors
    ax = df_emb.plot(x='Year', y='Value', kind='bar', legend=False, width=1.0, color=colors)

    plt.title('Cluster Labels')
    plt.xlabel('Year')
    plt.ylabel('')

    for label in ax.get_xticklabels():
        if int(label.get_text()) not in [1790, 1921, 2018]:
            label.set_visible(False)

    plt.yticks([])

    plt.show()

In [ ]:
#
# Clusters in UMAP 3D embedding
#
plot_clustering_diagram(umap_3_embedding)

A pandas dataframe where the temporal separation of clusters can be checked, and the watershed year(s) can be easily determined by inspection.

In [ ]:
#
# Dataframe with years labelled with 0 / 1 for clustering
#
def df_clustering(embedding):
    
    kmeans = KMeans(n_clusters=2, random_state=42)
    
    cluster_labels = kmeans.fit_predict(embedding)
    labels_year = [int(s.split('_')[1].split('.')[0]) for s in filenames]
    
    df_emb = pd.DataFrame(data=np.array([labels_year, cluster_labels]).T,
                          columns=['Year', 'Cluster'])
    df_emb = df_emb.sort_values('Year')
    
    return df_emb

In [ ]:
df_ripple_umap = df_clustering(umap_3_embedding)
df_ripple_umap

In [ ]:
#
# Clusters in TriMAP 3D embedding
#
plot_clustering_diagram(trimap_3_embedding)

In [ ]:
df_ripple_trimap = df_clustering(trimap_3_embedding)
df_ripple_trimap

In [ ]:
#
# Clusters in PaCMAP 3D embedding
#
plot_clustering_diagram(pacmap_3_embedding)

In [ ]:
df_ripple_pacmap = df_clustering(pacmap_3_embedding)
df_ripple_pacmap

In [ ]:
#
# Clusters in the initial BERT embedding
#
plot_clustering_diagram(bert_embedding)

In [ ]:
df_ripple_bert = df_clustering(bert_embedding)
df_ripple_bert

# Authorship attribution from embeddings

TOP1 and TOP5 nearest neighbors in the BERT embedding

In [ ]:
#
# Calculating TOP1 / TOP5 accuracy from nearest neighbours in the BERT embedding
#
def compute_accuracy(test_top_k):
    top_1_counter = 0
    top_k_counter = 0
    #
    n_categories = 1.0*len(test_top_k.keys())
    #
    for filename, results in test_top_k.items():
        X = filename.split('_')[1]
        n_tests = len(results)
        for res in results:
            y = res[0][0].split('_')[0]
            top_1_counter += 1.0*(X==y) / n_tests
            top_k_counter += any([X==y.split('_')[0] for y in res[0]]) / n_tests
    top_1_acc = top_1_counter/n_categories
    top_k_acc = top_k_counter/n_categories
    return top_1_acc, top_k_acc

In [ ]:
test_top_5 = test_text_files_in_sotu_directories(".", index, filenames, top_k=5)

In [ ]:
#
# TOP1 accuracy, TOP5 accuracy
#
top_1_acc, top_5_acc = compute_accuracy(test_top_5)
print(f'Accuracy: TOP1 {top_1_acc}, TOP5 {top_5_acc}')

TOP1 and TOP5 nearest neighbors after TriMAP: the Zeitgeist and general politics dominate authorship in temporal clustering

In [ ]:
trimap_index = faiss.IndexFlatL2(3)
trimap_index.add(trimap_3_embedding)
print(trimap_index.ntotal, len(filenames), trimap_index.is_trained)

In [ ]:
test_top_5_trimap = test_text_files_in_sotu_directories(".", trimap_index, filenames, top_k=5)

In [ ]:
#
# TOP1 accuracy, TOP5 accuracy
#
top_1_acc_trimap, top_5_acc_trimap = compute_accuracy(test_top_5_trimap)
print(f'Accuracy: TOP1 {top_1_acc_trimap}, TOP5 {top_5_acc_trimap}')